In [2]:
import pandas as pd

In [4]:
# We will load in the data from the dataset
df = pd.read_csv("../data/sales_dataset.csv")
df.head(5)

,order_date,order_number,customer_id,customer_name,zipcode,city,state_id,state_name,product_code,product_name,category,unit_price,unit_profit,quantity,revenue,profit,discount_percentage,discount_amount,lead_time_weeks
0,2019-11-13,Ord-3255406,cust-00004673,Ariana Nixon,32162,The Villages,FL,Florida,Nut-48306,Macadamia Style 5 Raw 11.34kg Bag,Nuts,26.26,9.19,3,78.77,27.57,0.0,0.0,2
1,2016-04-06,Ord-6058810,cust-00038703,Mohammad Esparza,80123,Littleton,CO,Colorado,Nut-48306,Macadamia Style 5 Raw 11.34kg Bag,Nuts,26.26,9.19,1,26.26,9.19,0.0,0.0,2
2,2019-08-15,Ord-1857958,cust-00106675,Jada Porter,55433,Minneapolis,MN,Minnesota,Nut-48306,Macadamia Style 5 Raw 11.34kg Bag,Nuts,26.26,9.19,4,105.02,36.76,0.0,0.0,2
3,2019-10-01,Ord-8598538,cust-00188381,Ari Mcgrath,36201,Anniston,AL,Alabama,Nut-48306,Macadamia Style 5 Raw 11.34kg Bag,Nuts,26.26,9.19,3,78.77,27.57,0.0,0.0,2
4,2016-07-24,Ord-3416282,cust-00213804,Kaitlynn Watkins,46143,Greenwood,IN,Indiana,Nut-48306,Macadamia Style 5 Raw 11.34kg Bag,Nuts,26.26,9.19,4,105.02,36.76,0.0,0.0,2


In [5]:
# For the time being we'll retain only what we believe to be essential for transportation
columns_to_drop = [
    "order_date",
    "order_number",
    "customer_id",
    "customer_name",
    "product_code",
    "category",
    "discount_percentage",
    "discount_amount",
    "lead_time_weeks"    
]
trans_df = df.drop(columns=columns_to_drop)
trans_df.head(5)

,zipcode,city,state_id,state_name,product_name,unit_price,unit_profit,quantity,revenue,profit
0,32162,The Villages,FL,Florida,Macadamia Style 5 Raw 11.34kg Bag,26.26,9.19,3,78.77,27.57
1,80123,Littleton,CO,Colorado,Macadamia Style 5 Raw 11.34kg Bag,26.26,9.19,1,26.26,9.19
2,55433,Minneapolis,MN,Minnesota,Macadamia Style 5 Raw 11.34kg Bag,26.26,9.19,4,105.02,36.76
3,36201,Anniston,AL,Alabama,Macadamia Style 5 Raw 11.34kg Bag,26.26,9.19,3,78.77,27.57
4,46143,Greenwood,IN,Indiana,Macadamia Style 5 Raw 11.34kg Bag,26.26,9.19,4,105.02,36.76


In [6]:
# It would be best to know weight rather than exclusively quantity for the purpose of calculation shipping costs.
# Not all product have weights set, let's determine the ones that don't and estimate a weight
weight_in_kg_regex = "(\d*.\d*)kg"
contains_kg_in_name = trans_df["product_name"].str.contains(pat=weight_in_kg_regex)
print(trans_df.loc[contains_kg_in_name==False]["product_name"].value_counts())

# As we can see, two items do not have a discernible weight, let's hardcode an estimate of 10kg for each
trans_df["unit_weight"] = trans_df["product_name"].str.extract(pat=weight_in_kg_regex)
trans_df["unit_weight"] = trans_df["unit_weight"].astype("float").fillna(10)


/var/folders/yf/d9s9jsm91sxf8bbpbmtqrfh80000gn/T/ipykernel_38233/2827583970.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  contains_kg_in_name = trans_df["product_name"].str.contains(pat=weight_in_kg_regex)


Plasctic Square Tub 102 Tubs per box PPWTUB       1273
Plastic Square Tub Lid 714 lids per box PPWLID    1269
Name: product_name, dtype: int64


In [9]:
# Let's now aggregate weight per city
trans_df["net_weight"] = trans_df["unit_weight"] * trans_df["quantity"]
trans_df["city_state"] = trans_df["city"] + "_" + trans_df["state_id"]

columns_to_drop = [
    "zipcode",
    "city",
    "state_id",
    "state_name",
    "product_name",
    "unit_price",
    "unit_profit",
    "revenue",
    "profit",
    "quantity",
    "unit_weight"
]
city_weight = trans_df.drop(columns=columns_to_drop)
city_weight_agg = city_weight.groupby('city_state').sum()

# Now we need to join the list of cities to their LAT, LONG values. We can pull this from US Census Data.
us_cities = pd.read_csv("../data/uscities.csv")
us_cities['city_state'] = us_cities["city_ascii"] + "_" + us_cities["state_id"]
us_cities = us_cities[['city_state', 'lat', 'lng']]
us_cities.head(5)

city_weight_agg = city_weight_agg.join(us_cities.set_index('city_state'), how="left")
city_weight_agg = city_weight_agg.groupby(['city_state']).mean(['lat', 'lng']).dropna()
city_weight_agg.head(5)


,net_weight,lat,lng
city_state,,,
Abbeville_LA,498.40,29.9751,-92.1265
Aberdeen_MD,80.00,39.5151,-76.1733
Aberdeen_SD,109.88,45.4649,-98.4686
Abilene_TX,7766.32,32.4543,-99.7384
Abingdon_MD,4564.60,39.4629,-76.2754


In [11]:
# Let's append the distances to the Dallas warehouse
import common

existing_wh = 'Dallas_TX'
common.append_distances_from_wh(city_weight_agg, existing_wh)

# We'll pickle the result to parallelize with
pickle_path = "../data/city_weight_agg.pkl"
city_weight_agg.to_pickle(pickle_path)


In [8]:
# # We'll keep a dummy version to play with in notebooks
# dummy = city_weight_agg.head(5).copy(deep=True)

# # Now we want to choose a city from the list of cities and consider it as the secondary warehouse
# # With this information we construct a list of distances to other cities
# # Let's first do this for one warehouse location as pet example. 
# # We arbitrarily choose a warehouse in Abbeville LA, and to keep things light, we'll pretend
# # the existing warehouse is in Aberdeen


# from common import append_distances_from_wh

# existing_wh = 'Aberdeen_MD'
# append_distances_from_wh(dummy, existing_wh)

# new_wh = 'Abingdon_MD'
# append_distances_from_wh(dummy, new_wh)

# # We'll establish the warehouse that's closest to the destination
# dummy['optimal_dist'] = dummy[['Aberdeen_MD_dist', 'Abingdon_MD_dist']].min(axis=1)
# dummy['from_aberdeen'] = (dummy['Aberdeen_MD_dist'] < dummy['Abingdon_MD_dist'])

# print(dummy.head(5))

# # # This gives a total of the number of cities served by each warehouse
# print(dummy.from_aberdeen.value_counts())

# # # We'll get the optimal kg*km sum as well
# dummy['weight_distance_shipped'] = dummy['optimal_dist'] * dummy['net_weight']
# optimal_weight_distance = dummy['weight_distance_shipped'].sum()
# print(optimal_weight_distance)

In [13]:
# After the parallelization has run we can analyze the contents of the
# weight_distances_log to see what the minimum weight_distance combination is

df = pd.read_csv("../data/weight_distances_log.csv")
df[df['weight_distance']==df['weight_distance'].min()]

,city_state,weight_distance,from_dallas,from_away
2511,Spencer_WV,1.023235e+10,1033,1605


In [18]:
df.sort_values('weight_distance')


,city_state,weight_distance,from_dallas,from_away
2511,Spencer_WV,1.023235e+10,1033,1605
299,Charleston_WV,1.024362e+10,1024,1614
330,Clarksburg_WV,1.024369e+10,1059,1579
1397,Fayetteville_WV,1.024438e+10,1034,1604
1358,Parkersburg_WV,1.024521e+10,1031,1607
...,...,...,...,...
731,Balch Springs_TX,1.539499e+10,880,1758
269,Cedar Hill_TX,1.540847e+10,2078,560
1918,Waxahachie_TX,1.541763e+10,2190,448
2221,Red Oak_TX,1.542455e+10,2244,394
